# Examining the rhyme scoring code

This notebook is mainly to provide more insight into the rhyme scoring algorithm. In the end, the scoring code has quite a few moving parts, and it wasn't practical to try and explain it in the paper, but the reviwers were keen to see the full details. Note that this code won't run standalone, I've just pulled out the core of the scoring code to explain how it works.



## Vowel similarity

First let's look at the implementation of the vowel similarity. This is simply based on the closeness of the vowels of Latin according to a 'normal' linguistic formant frequency chart. The vowels of Latin are below. Note that I do not differentiate between long and short vowels, which renders a considerable amount of controversy moot. Allen in _Vox Latina_ posits a system in which some long vowels are positioned differently to the short ones. Weiss and Calabrese more or less suggest a 5-vowel system (not including the Greek y in their analysis) and there is a good overview of the discussion on reddit (not exactly a scholarly source, but it's an efficient description by someone who clearly knows what they're talking about) [here](https://www.reddit.com/r/latin/comments/95yxez/vowel_pronunciation_beyond_allens_vox_latina/)

![Rhyme Vowel Similarity](rhyme_vowelsim.png)


In [1]:
# 10/11 bumped i-e slightly and o-a slightly based on
# Hirjee & Brown
NUCLEUS_SCORES = {
    "i": {"i": 1, "e": 0.75, "a": 0.5, "o": 0.42, "u": 0.4, "ü": 0.5},
    "e": {"i": 0.75, "e": 1, "a": 0.6, "o": 0.5, "u": 0.42, "ü": 0.5},
    "a": {"i": 0.5, "e": 0.6, "a": 1, "o": 0.6, "u": 0.42, "ü": 0.4},
    "o": {"i": 0.42, "e": 0.5, "a": 0.6, "o": 1, "u": 0.75, "ü": 0.35},
    "u": {"i": 0.4, "e": 0.42, "a": 0.42, "o": 0.75, "u": 1, "ü": 0.6},
    "ü": {"i": 0.5, "e": 0.5, "a": 0.4, "o": 0.35, "u": 0.6, "ü": 1},
}

## Consonant similarity

In standard rhyme, the syllable onsets are ignored, but the codas are important (ie 'bat' and 'cat' are perfect rhymes but 'kit' and 'kin' are not). Wherever consonants are important, we need to consider the quality of imperfect rhymes, so 'cut' and 'cup' are better than 'cut' and 'cuff'. In this implementation I only create one level of similarity, so two consonants are either identical, similar or dissimilar. The code below determines that similarity based on phonological features, but it is slightly complicated by the fact that, to my ear, some pairs that sound similar in an onset do not match as well in a coda. Finally, for the final syllable (always unstressed in Latin) I do consider the onset so that things like /ra.bit/ and /ra.bid/ can be upgraded due to the matching 'b'.

Essentially similar consonants just give a bonus to the rhyme score, but the exact details are a bit fiddly.

In [2]:
# Define a bunch of feature classes as sets. These are fairly standard phonological classes.

# fricatives
FRIC = {"s", "f", "z", "h"}

# stops, voiced / unvoiced
UNV_STOP = {"k", "t", "p"}
V_STOP = {"g", "d", "b"}
STOP = UNV_STOP | V_STOP

ALVEOLAR = {"t", "d", "s", "z"}
VELAR = {"g", "k"}
# bilabial
BILAB = {"p", "b", "w"}
# sonorant
SON = {"n", "m", "l", "r"}
# nasal
NAS = {"n", "m"}
# approximants
APPROX = {"j", "w", "l", "r"}
CONT = SON | NAS | FRIC | {""}

CONS_CLOSE = {
    "": FRIC | UNV_STOP | NAS | {""},
    "t": ALVEOLAR | STOP,
    "d": STOP,
    "s": FRIC | (UNV_STOP - BILAB),
    "f": FRIC,
    "k": STOP - BILAB,
    "h": STOP,  # only occurs as kh and th which are both stops
    "g": STOP - BILAB,
    "r": SON,
    "n": SON,
    "m": CONT,  # m isn't really there, it nasalises the vowel
    "l": SON,
    "b": (V_STOP | BILAB) - VELAR,  # b--g seems too far away
    "p": STOP - VELAR,
    "x": UNV_STOP | FRIC,
    "w": BILAB,
    "j": APPROX,
}

CLOSE_STRESSED_CODA = {
    "": FRIC | UNV_STOP,
    "b": STOP,
    "k": STOP,
    "d": STOP,
    "f": FRIC,
    "g": STOP,
    "h": STOP,  # only occurs in coda as kh and th which are both stops
    "j": APPROX,
    "l": SON,
    "m": SON,
    "n": SON,
    "p": STOP,
    "r": SON,
    "s": FRIC | (UNV_STOP - BILAB),
    "t": ALVEOLAR | (UNV_STOP - BILAB),
    "w": {"w"},  # should not appear in coda
    "x": {"x"},
}

CLOSE_FINAL_ONSET = {
    "b": STOP,
    "k": VELAR,
    "d": {"d", "t"},
    "f": FRIC,
    "g": VELAR,
    "h": FRIC,
    "j": APPROX,
    "l": {"r"},
    "m": NAS,
    "n": NAS,
    "p": STOP - VELAR,
    "r": {"l"},
    "s": FRIC | {"t"},
    "t": FRIC | {"k", "d", "r"},
    "w": APPROX,
    "x": {"x"},
    "": {""},
}

CLOSE_FINAL_CODA = {
    "b": V_STOP,
    "k": UNV_STOP,
    "d": V_STOP,
    "f": FRIC,
    "g": VELAR,
    "h": UNV_STOP,
    "j": {"j"},  # shouldn't happen
    "l": {"r"},
    "m": NAS | {" "},
    "n": NAS,
    "p": UNV_STOP,
    "r": {"l"},
    "s": FRIC | {"t"},
    "t": {"s", "p", "k", "d"},
    "w": {"w"},  # shouldn't happen
    "x": {"x"},
    "": {""},
}

## Nuclei

Score the a pair of syllables according to the nucleus. Diphthongs are allowed, and we score them according to the final position (ie 'ae' ends at 'e').

In [3]:
def _score_nucleus(s1, s2):
    if s1.nucleus == "" or s2.nucleus == "":
        return 0
    try:
        # Basic score for the final vowel
        nuc1 = s1.nucleus.translate(DEMACRON).lower()
        nuc2 = s2.nucleus.translate(DEMACRON).lower()
        v1 = s1.main_vowel
        v2 = s2.main_vowel
        score = NUCLEUS_SCORES[v1][v2]
        # print("Basic score for %s %s: %.2f" % (s1,s2,score))

        # One's a dipthong and one isn't, apply a penalty
        if len(nuc1) != len(nuc2):
            score *= 0.7
        elif (nuc1 != nuc2) and (v1 == v2):
            # two dipthongs but only last vowel equal
            score *= 0.7
        elif nuc1 == nuc2:
            # mismatched nasalisation:
            # if 1 (but not 0 or 2) of the nuclei is nasalised apply a small penalty
            if len([x for x in [s1.nucleus, s2.nucleus] if COMBINING_TILDE in x]) == 1:
                score *= 0.9
        else:
            # mismatched dipthongs or mismatched single letters
            score = score

    except Exception as e:
        print(s1)
        print(s2)
        raise e
    return score

## Syllable rhymes

Now two methods for calulating the rhyme for two syllables. The algorithm is slightly different for the stressed syllable as compared to the final syllable. Some words also have a mismatched number of syllables involved in the rhyme, which receives a penalty.

In [4]:
def _stressed_syl_rhyme(s1, s2):
    # onset doesn't matter, less fussy about 'r' in coda
    score = _score_nucleus(s1, s2)

    last1 = s1.coda[-1:].lower()
    last2 = s2.coda[-1:].lower()

    try:

        # perfect match receives a bonus
        if s1.coda == s2.coda:
            if s1.coda:
                score *= 1.2
            else:
                score *= 1

        elif len(s1.coda) + len(s2.coda) > 2:
            # at least one consonant cluster
            if "s" in s1.coda.lower() and "s" in s2.coda.lower():
                # ast as are close
                score *= 0.95
            elif (
                last2 in CLOSE_STRESSED_CODA[last1]
                or last1 in CLOSE_STRESSED_CODA[last2]
            ):
                # otherwise go by the final consonant - pakt part are close (?review?)
                score *= 0.9
            else:
                score *= 0.8

        elif last2 in CLOSE_STRESSED_CODA[last1] or last1 in CLOSE_STRESSED_CODA[last2]:
            score *= 0.95

        else:
            score *= 0.8

    except KeyError:
        score *= 0.8

    if score > 1:
        score = 1
    return score


def _final_syl_rhyme(s1, s2):

    # TODO move the magic score multipliers into a config dict
    
    # in the final syllable we apply a bonus
    # for matching onsets, stricter about codas
    score = _score_nucleus(s1, s2)

    first1 = s1.onset[0:1].lower()
    first2 = s2.onset[0:1].lower()

    try:
        if s1.onset == s2.onset:
            score *= 1.1

        elif len(s1.onset) + len(s2.onset) > 2:
            # at least one cluster
            if (
                first2 in CLOSE_FINAL_ONSET[first1]
                or first1 in CLOSE_FINAL_ONSET[first2]
            ):
                # otherwise go by the initial consonant - tra and ta are close (?review?)
                score *= 0.95
            else:
                score *= 0.85

        elif first2 in CLOSE_FINAL_ONSET[first1] or first1 in CLOSE_FINAL_ONSET[first2]:
            score *= 1

        else:
            score *= 0.85
    except KeyError:
        score *= 0.85

    last1 = s1.coda[-1:].lower()
    last2 = s2.coda[-1:].lower()

    try:

        # perfect match is good
        if s1.coda == s2.coda:
            if s1.coda:
                score *= 1.2
            else:
                score *= 1.1

        elif len(s1.coda) + len(s2.coda) > 2:
            # at least one cluster
            if "s" in s1.coda.lower() and "s" in s2.coda.lower():
                # ast as are close
                score *= 0.95
            elif (
                last2 in CLOSE_STRESSED_CODA[last1]
                or last1 in CLOSE_STRESSED_CODA[last2]
            ):
                # otherwise go by the final consonant - pakt part are close (?review?)
                score *= 0.9
            else:
                score *= 0.8

        elif last2 in CLOSE_STRESSED_CODA[last1] or last1 in CLOSE_STRESSED_CODA[last2]:
            score *= 0.95

        else:
            score *= 0.8

    except KeyError:
        score *= 0.8

    if score > 1:
        score = 1
    return score

def word_rhyme(w1, w2) -> (float):

    """Score the rhyme of two Words. Safe to call if one or
    both of the words are None (will return 0).

    Args:
        w1, w2 (rhyme_classes.Word): words to score

    Returns:
        (float): The score.
    """

    # This is so the user can call this with something
    # like l[-1] vs l.midword, where midword might not exist
    if not w1 or not w2:
        return 0

    # syls _might_ be empty, if the word is 'est' and it got eaten
    # by the previous word (prodelision)
    if len(w1.syls) == 0 or len(w2.syls) == 0:
        return 0

    if len(w1.syls) == 1 and len(w2.syls) == 1:
        s = _final_syl_rhyme(w1.syls[0], w2.syls[0])
        return s * 2

    # calculate the rhyme score on the stressed syllable
    stress_score = _stressed_syl_rhyme(w1.stressed_syllable, w2.stressed_syllable)
    score = stress_score

    # Now the rhyme on the remainder. In Latin, in theory,
    # the final syllable is never stressed, so there should be
    # at least one extra, but there _are_ exceptions.

    # For uneven lengths, if we have Xx vs Yyy then compare
    # the two final syllables, slurring over like
    # UN.də.ground // COM.pound
    coda_score = 0

    if len(w1.post_stress) > 0 and len(w2.post_stress) > 0:
        # single syllable words have their score doubled during
        # final_syl_rhyme
        coda_score = _final_syl_rhyme(w1.syls[-1], w2.syls[-1])

        # bump up really good final syllable matches. This biases the approach
        # somewhat since the final syllable is unstressed, but I have a pretty
        # strong intuition that this sort of final-syllable assonance/slant-rhyme
        # was important. On this see also Norberg (1968) 'Manuel pratique de Latin medieval'.
        # Norberg traces the development of medeval rhyme to final-syllable assonances
        # (some quite weak) in Sedulius in C4 CE, believing (as was common)
        # that classical rhyme was only accidental.
        if coda_score >= 0.75:
            coda_score *= 1.3

        # apply a small penalty for interstitial syllables between
        # stressed and final if there's a length mismatch
        # TODO: consider lightening this penalty. It was probably
        # routine to swallow these interstitials in 'normal' speech
        # and so perhaps too in poetry.
        # "Sed Augustus quoque in epistulis ad C. Caesarem
        # scriptis emendat quod is 'calidum' dicere quam 'caldum'
        # malit, non quia id non sit Latinum, sed quia sit odiosum"
        # (Quint. 1.6.19)
        if len(w1.post_stress) + len(w2.post_stress) == 3:
            # a 1 and a 2. This will be 99% of the cases. If it's
            # not this then something weird is happening and the
            # rest of the logic here might break.
            longer = max(w1.post_stress, w2.post_stress, key=len)
            # mid-low vowels (e,a,o) get pronounced as a schwa in the interstitial syllable
            # but high ones (i,u,ü) sound more obtrusive to me.
            if (
                len(longer[1].nucleus.translate(DEMACRON).lower()) > 1
                or longer[1].main_vowel in "iuü"
            ):
                coda_score *= 0.73
            else:
                coda_score *= 0.83

        score += coda_score

    return score

# Scoring some words

Here I'll just run through the kind of code used to produce Table 1 (the list of example rhyme scores)

In [5]:
from mqdq import rhyme, babble

import random
import string
import scipy as sp
import pandas as pd

In [11]:
met_single_bab = babble.Babbler.from_file('mqdq/OV-meta.xml', name='Metamorphoses')

In [12]:
# this is now how I would normally syllabify, but if we want to examine
# individual word rhymes we need to take them before applying elision,
# prodelision etc. The 'normal' system calculates rhyme for the line
# as pronounced, ie if 'tua est' is at the end of a line the 'final' word
# is tuast, NOT est.

words = []
for l in met_single_bab.raw_source:
    a = [rhyme._phonetify(rhyme._syllabify_word(x)) for x in l('word')]
    words.extend(a)

In [13]:
# Collect 25 random pairs of words whose rhyme score is 
# above 1.75 (the global threshhold used in all the experiments)

pairs = []
while len(pairs) < 25:
    w1, w2 = random.sample(words, 2)
    a = w1.mqdq.text.translate(str.maketrans('', '', string.punctuation)).lower()
    b = w2.mqdq.text.translate(str.maketrans('', '', string.punctuation)).lower()
    if a==b:
        continue
    score, ss, cs = rhyme._word_rhyme_debug(w1,w2)
    if 1.75 <= score:
        pairs.append((w1,w2,(score,ss,cs)))

In [14]:
def table(pairs):
    res = []
    for p in pairs:
        score = p[2][0]
        syls1 = ('.'.join(p[0].syls)).lower()
        syls2 = ('.'.join(p[1].syls)).lower()
        w1 = p[0].mqdq.text.translate(str.maketrans('', '', string.punctuation)).lower()
        w2 = p[1].mqdq.text.translate(str.maketrans('', '', string.punctuation)).lower()
        row = {
            'orth1': w1,
            'orth2': w2,
            'phon1': syls1,
            'phon2': syls2,
            'score': score,
            'stress': p[2][1],
            'final': p[2][2],
        }
        res.append(row)
    return pd.DataFrame(res)

In [15]:
# Max possible score is 2.30.

table(pairs).sort_values(by='score')

,orth1,orth2,phon1,phon2,score,stress,final
4,sola,auras,`sō.la,`au.ras,1.760000,0.5250,1.235000
10,dapes,lyramque,`da.pes,lü.`ram.kwe,1.769750,0.7200,1.049750
15,sequentes,phrygiisque,se.`kwen.tes,prü.gi.`īs.kwe,1.773250,0.6000,1.173250
14,pueri,corpore,`pu.e.rī,`kor.po.re,1.779750,0.6000,1.179750
13,erit,foret,`e.rit,`fo.ret,1.787000,0.5000,1.287000
24,debita,persequar,`dē.bi.ta,`per.se.kwar,1.788000,0.8000,0.988000
8,flere,frustraque,`fle.re,frus.`tra.kwe,1.815500,0.6000,1.215500
21,aquosis,aonides,a.`kwo.sis,ā.`o.ni.des,1.825435,1.0000,0.825435
1,agitat,robora,`a.gi.tat,`rō.bo.ra,1.835000,0.6000,1.235000
0,corrigit,potui,`kor.ri.git,`po.tu.ī,1.849750,0.8000,1.049750


# Future Work

The scoring system seems 'good enough' to me in that it mostly captures 'rhymes' (which I use to mean interesting sonic correspondences) and mostly rejects uninteresting pairs. The ordering of scores can be a bit flaky, so it would be good to improve that at some point. Several reveiwers have expressed concern that ignoring vowel lengths sometimes causes pairs that score too highly. It would be great to reflect spoken vowel length, but it is a little tricky when we have vowels that lengthen to 'make position' (which technical Latin poetry thing)--it is not certain how those vowels were pronounced, all I would be able to say for sure is how the vowels were _scanned_. At that point we would need to sort through the phonological debate between 'Allen style' and 'Calabrese style' pronunciation constructions, which is not something I look forward to with extreme pleasure.